In [ ]:
#THIS PROGRAM TAKES IN A CSV FILE WITH A LIST OF STOCK
#ITERATES THROUGH THOSE STOCKS AND GATHERS THE PREVIOUS MONTHS DATA AND SAVES TO GOOGLE SHEETS
#USING BARCHART API

In [1]:
# IMPORTS #
import time
t0 = time.clock()

import pandas as pd
from pandas.tseries.offsets import BDay
import numpy as np
import datetime as dt
from copy import copy
import warnings

warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)


## GOOGLE SHEETS API

In [2]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# use creds to create a client to interact with the Google Drive API
scope = ['https://spreadsheets.google.com/feeds', 
            'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope)
client = gspread.authorize(creds)

# Find a workbook by name and open the first sheet
# Make sure you use the right name here.
stockList = client.open("STOCKS LIST").sheet1
AAPL_data = client.open("AAPL_data").sheet1
VZ_data = client.open("VZ_data").sheet1
AMZN_data = client.open("AMZN_data").sheet1
MSFT_data = client.open("MSFT_data").sheet1
TSLA_data = client.open("TSLA_data").sheet1

GOOGL_data = client.open("GOOGL_data").sheet1
INTC_data = client.open("INTC_data").sheet1
CSCO_data = client.open("CSCO_data").sheet1
ORCL_data = client.open("ORCL_data").sheet1
QCOM_data = client.open("QCOM_data").sheet1
FB_data = client.open("FB_data").sheet1
IBM_data = client.open("IBM_data").sheet1


# Extract and print all of the values
list_of_hashes = stockList.get_all_records()
print(list_of_hashes)
#setUpSheetsClient()

[{'STOCKS': 'AAPL'}, {'STOCKS': 'MSFT'}, {'STOCKS': 'AMZN'}, {'STOCKS': 'VZ'}, {'STOCKS': 'TSLA'}, {'STOCKS': 'GOOGL'}, {'STOCKS': 'INTC'}, {'STOCKS': 'CSCO'}, {'STOCKS': 'ORCL'}, {'STOCKS': 'QCOM'}, {'STOCKS': 'FB'}, {'STOCKS': 'IBM'}]


In [ ]:
# ================================================================== #
# filepath management

project_dir = r'/Users/jamesgalante/Documents/College/Semester 7/CPEG498/Uptick/Legacy Data/get past month data/' 
price_path = project_dir + r'Symbols/'
apikey = '0adbf00b462c1acca954a43d94279b92'


In [ ]:
##points to our google sheet "STOCKS LIST" gets the values and makes it a pandas dataframe to manipulate
def getStockList():
    sheetsList = stockList.get_all_values()
    sheetsList = sheetsList[1:] #remove "STOCKS" title
    ##print(sheetsList)
    sheetsList = flatten(sheetsList)
    #print(sheetsList)
    #create new df 
    df = pd.DataFrame({'STOCKS':sheetsList})
    return df


In [ ]:
##takes a 2d list and flattens it to 1d
def flatten(input):
    new_list = []
    for i in input:
        for j in i:
            new_list.append(j)
    return new_list

In [ ]:
#setting list of stock symbols to var syms
syms = getStockList()
print(syms)

In [ ]:
#BARCHART API
def construct_barChart_url(sym, start_date, freq, interval, api_key=apikey):
    '''Function to construct barchart api url'''
    
    url = 'http://marketdata.websol.barchart.com/getHistory.csv?' +\
            'key={}&symbol={}&type={}&startDate={}&interval={}'.format(api_key, sym, freq, start_date, interval)
    return url

In [ ]:
#get minute data for the past month for the stocks in the csv file
def get_minute_data(start):
    print("minute data:")
    '''Function to Retrieve <= 3 months of minute data for SP500 components'''
    
    # This is the required format for datetimes to access the API
    # You could make a function to translate datetime to this format
    #start = '20181008' #start data doesnt matter as long as its more then a month older
    #end = d
    freq = 'minutes'   
    interval = 10 #minutes
    prices = {}
    symbol_count = len(syms)
    N = copy(symbol_count)
    try:
        for i, sym in syms.iterrows():
            
            print(sym["STOCKS"])
            api_url = construct_barChart_url(sym["STOCKS"], start, freq, interval , api_key=apikey)
            print(api_url)
            try:
                csvfile = pd.read_csv(api_url, parse_dates=['timestamp'])
                csvfile.set_index('timestamp', inplace=True)
                prices[sym["STOCKS"]] = csvfile
            except:
                continue
            N -= 1
            pct_total_left = (N/symbol_count)
            print('{}..[done] | {} of {} symbols collected | percent remaining: {:>.2%}'.format(\
                                                                sym, i, symbol_count, pct_total_left)) 
    except Exception as e: 
        print(e)
    finally:
        pass
    px = pd.Panel.from_dict(prices)

    return px



In [ ]:
def getStockDataFromDate(start):
    print(syms)
    pxx = get_minute_data(start)
    # convert timestamps to EST
    pxx.major_axis = pxx.major_axis.tz_localize('utc').tz_convert('US/Eastern')
    return pxx

In [ ]:
#sends pxx stock data to google sheets. will append pxx to gsheets
def sendLocalStockDataToSheets(pxx):
    print("Sending to google sheets")
    #curr_stock = AAPL_data
    for i, sym in syms.iterrows(): #iterate through the panel symbols
        for index, row in pxx[sym['STOCKS']].iterrows(): #iterate through each row in each symbol
            try:
                time.sleep(0.21)
                print("adding Row")
                if(row.symbol == 'AAPL'):
                    AAPL_data.append_row([str(row.name), row.symbol, row.tradingDay, row.open, row.high, row.low, row. close, row.volume])
                elif (row.symbol == 'VZ'):
                    VZ_data.append_row([str(row.name), row.symbol, row.tradingDay, row.open, row.high, row.low, row. close, row.volume])
                elif (row.symbol == 'TSLA'):
                    TSLA_data.append_row([str(row.name), row.symbol, row.tradingDay, row.open, row.high, row.low, row. close, row.volume])
                elif (row.symbol == 'AMZN'):
                    AMZN_data.append_row([str(row.name), row.symbol, row.tradingDay, row.open, row.high, row.low, row. close, row.volume])
                elif (row.symbol == 'MSFT'):
                    MSFT_data.append_row([str(row.name), row.symbol, row.tradingDay, row.open, row.high, row.low, row. close, row.volume])
                elif (row.symbol == 'IBM'):
                    IBM_data.append_row([str(row.name), row.symbol, row.tradingDay, row.open, row.high, row.low, row. close, row.volume])
                elif (row.symbol == 'FB'):
                    FB_data.append_row([str(row.name), row.symbol, row.tradingDay, row.open, row.high, row.low, row. close, row.volume])
                elif (row.symbol == 'QCOM'):
                    QCOM_data.append_row([str(row.name), row.symbol, row.tradingDay, row.open, row.high, row.low, row. close, row.volume])
                elif (row.symbol == 'ORCL'):
                    ORCL_data.append_row([str(row.name), row.symbol, row.tradingDay, row.open, row.high, row.low, row. close, row.volume])
                elif (row.symbol == 'CSCO'):
                    CSCO_data.append_row([str(row.name), row.symbol, row.tradingDay, row.open, row.high, row.low, row. close, row.volume])
                elif (row.symbol == 'INTC'):
                    INTC_data.append_row([str(row.name), row.symbol, row.tradingDay, row.open, row.high, row.low, row. close, row.volume])
                elif (row.symbol == 'GOOGL'):
                    GOOGL_data.append_row([str(row.name), row.symbol, row.tradingDay, row.open, row.high, row.low, row. close, row.volume])
                    
            except:
                print("Error adding row!", row)
                continue
    


In [ ]:
#updates the google sheets tables
def updateSheetsWithLatest():
    print("updating:")
    serverData = AAPL_data.get_all_values() #only gets the date of AAPL since they should all be uniform
    lastRowOnServer = serverData[-1]
    lastTimestampOnServer = lastRowOnServer[2]
    lastTimestampOnServer_noDash = lastTimestampOnServer.replace("-", "") #remove the dast
    
    localData = getStockDataFromDate(lastTimestampOnServer_noDash) 
    
    #localData = getStockDataFromDate('20180901') 

    print(type(localData))
 #   localData['AAPL'] = localData['AAPL'][localData['AAPL'].tradingDay != lastTimestampOnServer] #removes any duplicate
 #   localData['MSFT'] = localData['MSFT'][localData['MSFT'].tradingDay != lastTimestampOnServer] #removes any duplicate
 #   localData['AMZN'] = localData['AMZN'][localData['AMZN'].tradingDay != lastTimestampOnServer] #removes any duplicate
 #   localData['VZ'] = localData['VZ'][localData['VZ'].tradingDay != lastTimestampOnServer] #removes any duplicate
 #   localData['TSLA'] = localData['TSLA'][localData['TSLA'].tradingDay != lastTimestampOnServer] #removes any duplicate
 #   localData['IBM'] = localData['IBM'][localData['IBM'].tradingDay != lastTimestampOnServer] #removes any duplicate
 #   localData['FB'] = localData['FB'][localData['FB'].tradingDay != lastTimestampOnServer] #removes any duplicate
 #   localData['QCOM'] = localData['QCOM'][localData['QCOM'].tradingDay != lastTimestampOnServer] #removes any duplicate
 #   localData['ORCL'] = localData['ORCL'][localData['ORCL'].tradingDay != lastTimestampOnServer] #removes any duplicate
 #   localData['CSCO'] = localData['CSCO'][localData['CSCO'].tradingDay != lastTimestampOnServer] #removes any duplicate
 #   localData['INTC'] = localData['INTC'][localData['INTC'].tradingDay != lastTimestampOnServer] #removes any duplicate
 #   localData['GOOGL'] = localData['GOOGL'][localData['GOOGL'].tradingDay != lastTimestampOnServer] #removes any duplicate

    
    sendLocalStockDataToSheets(localData)

In [ ]:

def main():
    #getStockDataFromDate('20180932')
    updateSheetsWithLatest() 
    #sendLocalStockDataToSheets()
main()

In [3]:
i = 1
print("this is i:",i)

this is i: 1
